# Lazy vs Eager

Understand the difference between eager and lazy graph tracing and transpilation.

⚠️ If you are running this notebook in Colab, you will have to install `Ivy` and some dependencies manually. You can do so by running the cell below ⬇️

If you want to run the notebook locally but don't have Ivy installed just yet, you can check out the [Get Started section of the docs.](https://unify.ai/docs/ivy/overview/get_started.html)

In [1]:
!pip install ivy

`ivy.unify`, `ivy.trace_graph` and `ivy.transpile` can all be performed either eagerly or lazily. All previous examples have been performed **lazily**, which means that the unification, tracing, or transpilation process actually occurs during the first call of the **returned** function. 

This is because all three of these processes depend on function tracing, which requires function arguments to use for the tracing. Alternatively, the arguments can be provided during the `ivy.unify`, `ivy.trace_graph` or `ivy.transpile` call itself, in which case the process is performed **eagerly**. We show some simple examples for each case below.

## Unify

Consider again this simple `torch` function:

In [2]:
import ivy
import torch

def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

And let's also create the dummy `numpy` arrays as before:

In [3]:
# import NumPy
import numpy as np
np.random.seed(0)

# create random numpy array for testing
x = np.random.uniform(size=10)

Let's assume that our target framework is `tensorflow`:

In [4]:
import tensorflow as tf
ivy.set_backend("tensorflow")

x = tf.constant(x)

2023-11-01 06:53:37.201733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/workspaces/ivy/ivy/utils/exceptions.py:390: UserWarning: The current backend: 'tensorflow' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(


In the example below, the function is unified **lazily**, which means the first function call will execute slowly, as this is when the unification process actually occurs.

In [5]:
norm = ivy.unify(normalize, source="torch")
norm(x) # slow, lazy unification
norm(x) # fast, unified on previous call

ivy.array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])

However, in the following example the unification occurs **eagerly**, and both function calls will be fast:

In [6]:
ivy.set_backend("tensorflow")
norm = ivy.unify(normalize, source="torch", args=(x,))
norm(x) # fast, unified at ivy.unify
norm(x) # fast, unified at ivy.unify

ivy.array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])

## Trace

The same is true for tracing. In the example below, the function is traced **lazily**, which means the first function call will execute slowly, as this is when the tracing process actually occurs.

In [7]:
norm_trace = ivy.trace_graph(norm)
norm_trace(x) # slow, lazy graph tracing
norm_trace(x) # fast, traced on previous call

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

However, in the following example the tracing occurs **eagerly**, and both function calls will be fast:

In [8]:
norm_tracing = ivy.trace_graph(norm, args=(x,))
norm_tracing(x) # fast, traced at ivy.trace_graph
norm_tracing(x) # fast, traced at ivy.trace_graph

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

## Transpile

The same is true for transpiling. In the example below, the function is transpiled **lazily**, which means the first function call will execute slowly, as this is when the transpilation process actually occurs.

In [9]:
norm_trans = ivy.transpile(normalize, source="torch", to="tensorflow")
norm_trans(x) # slow, lazy transpilation
norm_trans(x) # fast, transpiled on previous call

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

However, in the following example the transpilation occurs *eagerly*, and both function calls will be fast:

In [10]:
norm_trans = ivy.transpile(normalize, source="torch", to="tensorflow", args=(x,))
norm_trans(x) # fast, transpiled at ivy.transpile
norm_trans(x) # fast, transpiled at ivy.transpile

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

## Round Up

That's it, you now know the difference between lazy vs eager execution for `ivy.unify`, `ivy.trace_graph` and `ivy.transpile`! Next, we'll be exploring how these three functions can all be called as function decorators!